In [ ]:
from bs4 import BeautifulSoup
import requests
import time, os
import pickle

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
def get_house_info(url, chromedriver):
    ''' Function that takes in the url of a sold home page and returns all house info in a dictionary'''
    time.sleep(3)
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    house_facts = {}
    
    if len(soup.find_all(class_="row PropertyHistoryEventRow")) != 0:
        house_facts['Beds'] = soup.find(class_="facts-table").find(text="Beds").next.text
        if house_facts['Beds'] == "—":
            try:
                house_facts['Beds']= soup.find(text = "Beds").parent.parent.find(class_="statsValue").text
            except: gft
                house_facts['Beds']= soup.find(text = "Bed").parent.parent.find(class_="statsValue").text
        house_facts['Baths']= soup.find(class_="facts-table").find(text="Baths").next.text
        if house_facts['Baths'] == "—":
            try:
                house_facts['Baths']= soup.find(text = "Baths").parent.parent.parent.parent.find(class_="statsValue").text
                if "$" in house_facts['Baths']:
                    house_facts['Baths']= soup.find(text = "Baths").parent.parent.find(class_="statsValue").text

            except: 
                house_facts['Baths']= soup.find(text = "Bath").parent.parent.parent.parent.find(class_="statsValue").text
                if "$" in house_facts['Baths']:
                    house_facts['Baths']= soup.find(text = "Bath").parent.parent.find(class_="statsValue").text

        house_facts['Sq. Ft.']= soup.find(class_="facts-table").find(text="Sq. Ft.").next.text
        if house_facts['Sq. Ft.'] == "—":
            house_facts['Sq. Ft.']= soup.find(class_ = "stat-block sqft-section").find(class_="statsValue").text
        house_facts['Year Built'] = soup.find(class_="facts-table").find(text="Year Built").next.text
        if house_facts['Year Built'] == "—":
            house_facts['Year Built'] = soup.find(text="Year Built").next.text
        house_facts['Lot Size'] = soup.find(class_="facts-table").find(text="Lot Size").next.text
        house_facts['Style'] = soup.find(class_="facts-table").find(text="Style").next.text
        house_facts['Stories'] = soup.find(class_="facts-table").find(text="Stories").next.text
        house_facts['Year Renovated'] = soup.find(class_="facts-table").find(text="Year Renovated").next.text
        house_facts['County'] = soup.find(class_="facts-table").find(text="County").next.text


        house_facts['sold_date'] = soup.find(class_="sold-row row PropertyHistoryEventRow").find(class_="col-4").text[:-4]
        house_facts['sold_price'] = soup.find(class_="sold-row row PropertyHistoryEventRow").find(class_="price-col number").text.strip()
        for i in soup.find_all(class_="row PropertyHistoryEventRow"):
            if "Listed" in i.find(class_="description-col col-4").find('div').text:
                house_facts['listed_price'] = i.find(class_="price-col number").text
                house_facts['listed_date'] = i.find(class_="col-4").text[:-4]
                break 

        try:
            house_facts['Community'] = soup.find(text = "Community").parent.parent.find(class_='content text-right').text
        except:
            try:
                house_facts['Community'] = soup.find(text = "Neighborhood").parent.parent.text[:-12]
            except:
                house_facts['Community'] = "—"
        try:
            house_facts['Redfin_estimate'] = soup.find(text = "Redfin Estimate").parent.parent.parent.parent.find(class_="content text-right").text
        except:
            house_facts['Redfin_estimate'] = "NA"
        try:
            house_facts['Price_sqft'] = soup.find(text = "Price/Sq.Ft.").parent.parent.find(class_="content text-right").text
        except:
            house_facts['Price_sqft'] = "NA"

        for index, i in enumerate([i.text for i in soup.find(class_="activityStatsTable").find_all(class_="label")]):
            if i == "all-time":
                house_facts['all-time-{}'.format([i.text for i in soup.find(class_="activityStatsTable").find_all(class_="label")][index-1])] = [i.text for i in soup.find(class_="activityStatsTable").find_all(class_="count")][index]
            else:
                house_facts[i] = [i.text for i in soup.find(class_="activityStatsTable").find_all(class_="count")][index]

        house_facts['school_ratings'] = [i.text for i in soup.find_all(class_="rating-num")]
        driver.close()
        
        return house_facts
    else:
        return 0
    

In [ ]:
def get_homes_data(url, chromedriver):
    '''Function that takes in a url for main page with all sold homes in an area, then navigates to individual homes pages
    and call get_house_info function to get house info. Function returns a list of dictionaries with homes info. 
    Function breaks when target number of houses data is achieved'''
    houses_data = []
    errors = []
    for i in range(1,4000):
        if len(houses_data) > 1500:
            break
        else:
            if i == 1:
                driver = webdriver.Chrome(chromedriver)
                driver.get(url)
                time.sleep(5)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                for house in soup.find_all( class_='v2 interactive'):
                    house_url = "https://www.redfin.com" + house.find('a')['href']
                    try:
                        house_info = get_house_info(house_url, chromedriver)
                    except:
                            errors.append(house_url)
                    if house_info != 0:
                        houses_data.append(house_info)
                driver.close()
                with open('datalist.pkl', 'wb') as f:
                    pickle.dump(houses_data, f)
            else:
                driver = webdriver.Chrome(chromedriver)
                driver.get(url + "/page-{}".format(i))
                time.sleep(5)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                for house in soup.find_all( class_='v2 interactive'):
                    house_url = "https://www.redfin.com" + house.find('a')['href']
                    try:
                        house_info = get_house_info(house_url, chromedriver)
                    except:
                            errors.append(house_url)
                    if house_info != 0:
                        houses_data.append(house_info)
                driver.close()
                with open('datalist.pkl', 'wb') as f:
                    pickle.dump(houses_data, f)

    return houses_data   

In [ ]:
chromedriver = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [ ]:
#get homes info
houses_data = get_homes_data("https://www.redfin.com/city/16163/WA/Seattle/recently-sold", chromedriver)

In [ ]:
#save final dataset
with open('datalist.pkl', 'wb') as f:
    pickle.dump(houses_data, f)